[opencv 教學](https://blog.gtwang.org/programming/opencv-webcam-video-capture-and-file-write-tutorial/)

In [1]:
# import face_recognition_api
import face_recognition.api as face_recognition_api
import cv2
import os
import pickle
import numpy as np
import warnings
from PIL import Image, ImageDraw, ImageFont
import time

In [2]:
# Load Face Recogniser classifier
fname = 'classifier.pkl'
if os.path.isfile(fname):
    with open(fname, 'rb') as f:
        (le, clf) = pickle.load(f)
else:
    print('\x1b[0;37;43m' + "Classifier '{}' does not exist".format(fname) + '\x1b[0m')
    quit()


In [3]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True


# video = 'D:\\data\\video\\綜藝大集合 20170709 台南 安南.mp4'
# video = 'D:\\data\\video\\綜藝大集合20180930 新竹新豐 - 片段1.mp4'
# video = 'D:\\data\\video\\綜藝大集合20180930 新竹新豐.mp4'
video = '綜藝大集合20180930 新竹新豐 - 片段1.mp4'
 

FILE_OUTPUT = video+'.output.avi'
print(FILE_OUTPUT)

綜藝大集合20180930 新竹新豐 - 片段1.mp4.output.avi


# 直接輸出到影像檔案中(存檔)

In [4]:
# Checks and deletes the output file
# You cant have a existing file or it will through an error
if os.path.isfile(FILE_OUTPUT):
    os.remove(FILE_OUTPUT)
    print('remove output file')

#####################
stime = time.localtime()
print(time.strftime("%Y-%m-%d %H:%M:%S",stime))
#####################
    
# Playing video from file:
# cap = cv2.VideoCapture(video)
cap = cv2.VideoCapture(0)
currentFrame, j = 0, 2
# Get current width of frame
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
# Get current height of frame
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # float
print('width, height',width, height)

fps = cap.get(cv2.CAP_PROP_FRAME_COUNT) # float
print(fps)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(FILE_OUTPUT,fourcc, 29.0, (int(width),int(height)))


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
# while(True):
    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()

        if ret == True:
            if currentFrame % j == 0:
#             if process_this_frame:    
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25, interpolation=cv2.INTER_CUBIC)
#                 small_frame = cv2.resize(frame, (0, 0), fx=0.2, fy=0.2, interpolation=cv2.INTER_CUBIC)
#                 small_frame = cv2.resize(frame, (0, 0), fx=1.0, fy=1.0, interpolation=cv2.INTER_CUBIC)
                print('currentFrame : ', currentFrame)
#                 face_locations = face_recognition_api.face_locations(small_frame, model='cnn')
                # 一次只能處理一張圖，所以不能做多線程，因為串流一次吃進一張圖處理完後輸出
                # number_of_times_to_upsample 這只能調整框臉的數量(數字越大可框出越多小臉)
                face_locations = face_recognition_api.face_locations(small_frame, number_of_times_to_upsample=1)
                face_encodings = face_recognition_api.face_encodings(small_frame, face_locations)  # 這很慢
            
                face_names = []
                predictions = []
#                 predictions = face_locations
                if len(face_encodings) > 0:
                    closest_distances = clf.kneighbors(face_encodings, n_neighbors=1)
                    is_recognized = [closest_distances[0][i][0] <= 0.3999 for i in range(len(face_locations))]
                
                    predictions = [(le.inverse_transform([int(pred)])[0], loc) if rec else ("　　　", loc) for \
                            pred, loc, rec in zip(clf.predict(face_encodings), face_locations, is_recognized)]
                    
                    print('predictions ',predictions)
                    print('closest_distances ',closest_distances)
#             process_this_frame = not process_this_frame
    
            for name, (top, right, bottom, left) in predictions:
#             for (top, right, bottom, left) in predictions:
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                if name != "　　　":
#                     cv2.rectangle(frame, (left, bottom - 30), (right, bottom), (0, 0, 255), cv2.FILLED)
#                     pil_im = Image.fromarray(frame)
#                     draw = ImageDraw.Draw(pil_im)
#                     font = ImageFont.truetype('C:\\Windows\\Fonts\\mingliu.ttc', 30, encoding='utf-8')   #, encoding='utf-8'
#                     draw.text((left, bottom - 30), name, (255, 255, 255), font=font)
#                     frame = np.array(pil_im)
                    
                    
#                     cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    cv2.rectangle(frame, (left, bottom - 30), (right, bottom), (0, 0, 255), cv2.FILLED)
                    pil_im = Image.fromarray(frame)
                    draw = ImageDraw.Draw(pil_im)
                    font = ImageFont.truetype('C:\\Windows\\Fonts\\simsun.ttc',30,encoding='utf-8')
                    draw.text((left, bottom - 30), name, (255, 255, 255), font=font)
                    frame = np.array(pil_im)
                    
                    
                else:
                    pass
                

#                     predictions = []
            

            # Saves for video
            out.write(frame)

            # Display the resulting frame
            cv2.imshow('frame',frame)
        else:
            break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # To stop duplicate images
        currentFrame += 1

# When everything done, release the capture
print('process finished on Frame ...' , currentFrame)
cap.release()
out.release()
cv2.destroyAllWindows()

etime = time.localtime()
print(time.strftime("%Y-%m-%d %H:%M:%S",etime))

remove output file
2019-02-23 16:50:37
width, height 1280.0 720.0
310.0
currentFrame :  0
predictions  [('胡瓜 Hu gua', (53, 190, 115, 128)), ('\u3000\u3000\u3000', (113, 320, 156, 280)), ('\u3000\u3000\u3000', (89, 261, 132, 218)), ('\u3000\u3000\u3000', (46, 289, 82, 253))]
closest_distances  (array([[0.34659386],
       [0.56768157],
       [0.55553858],
       [0.65732639]]), array([[0],
       [4],
       [4],
       [5]], dtype=int64))
currentFrame :  2
predictions  [('胡瓜 Hu gua', (53, 197, 115, 135)), ('\u3000\u3000\u3000', (89, 256, 132, 213)), ('\u3000\u3000\u3000', (118, 319, 161, 275)), ('\u3000\u3000\u3000', (70, 320, 106, 289))]
closest_distances  (array([[0.35032742],
       [0.56787922],
       [0.56825894],
       [0.57585776]]), array([[ 0],
       [ 4],
       [10],
       [10]], dtype=int64))
currentFrame :  4
predictions  [('胡瓜 Hu gua', (53, 204, 115, 142)), ('\u3000\u3000\u3000', (122, 313, 158, 277)), ('\u3000\u3000\u3000', (94, 249, 130, 213)), ('\u3000\u3000\u3000

In [5]:
# face_recognition_api.face_locations?